In [1]:
import pandas as pd
import pyodbc
from tqdm import tqdm
import dask.dataframe as dd
import polars as pl

conn=pyodbc.connect(
    Trusted_Connection=r'Yes',
    Driver=r'{ODBC Driver 17 for SQL Server}',
    Server=r'DESKTOP-QQPNCCN\SQLEXPRESS',
    Database=r'SQL Tutorial')

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Pandas

In [6]:
def read_sql_query_in_chunks(query, conn, chunksize=100000):
    total_rows = 0
    for chunk in tqdm(pd.read_sql_query(query, conn, chunksize=chunksize), unit=" chunk"):
        total_rows += len(chunk)
        if total_rows % 1000000 == 0:
            print(f"Read {total_rows} rows")
        yield chunk

query="Select * From [new].dbo.lung_data" 

df=pd.concat(read_sql_query_in_chunks(query, conn, chunksize=100000))

10 chunk [00:13,  1.32s/ chunk]

Read 1000000 rows


20 chunk [00:26,  1.33s/ chunk]

Read 2000000 rows


30 chunk [00:39,  1.33s/ chunk]

Read 3000000 rows


33 chunk [00:43,  1.30s/ chunk]


In [4]:
df.info(verbose = False, memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
Index: 3250000 entries, 0 to 49999
Columns: 18 entries, id to survived
dtypes: bool(6), float64(2), int64(2), object(8)
memory usage: 1.4 GB


## Dask

In [3]:
uri = (r'mssql+pyodbc:///?odbc_connect=DRIVER={ODBC Driver 17 for SQL Server};'
       r'SERVER=DESKTOP-QQPNCCN\SQLEXPRESS;DATABASE=new;Trusted_Connection=yes;')

# Specify the table name and index column
table_name = "lung_data"
index_col = "id" #Column which becomes the index, and defines the partitioning

# Create a Dask dataframe by reading from SQL table
ddf = dd.read_sql_table(table_name, uri, index_col=index_col,npartitions=10)

In [4]:
memory_usage = ddf.memory_usage(deep=True).compute().sum()
print(f"Memory usage: {round(memory_usage / (1024 ** 2),2)} MB")

Memory usage: 540.99 MB


In [7]:
dask_df = dd.from_pandas(df, npartitions=10)
memory_usage = dask_df.memory_usage(deep=True).compute().sum()
print(f"Memory usage: {round(memory_usage / (1024 ** 2),2)} MB")

Memory usage: 565.79 MB


## Polars

In [ ]:
def read_sql_query_in_batches(query, conn, batch_size=100000):
    total_rows = 0
    for batch in tqdm(pl.read_database(query,conn,batch_size=100000,iter_batches=True), unit=" batch"):
        total_rows += len(batch)
        if total_rows % 1000000 == 0:
            print(f"Read {total_rows} rows")
        yield batch
query="Select * From [new].dbo.lung_data" 


pdf = pl.concat(read_sql_query_in_batches(query, conn, batch_size=100000))

10 batch [00:14,  1.45s/ batch]

Read 1000000 rows


11 batch [00:16,  1.45s/ batch]

In [ ]:
print(f"Memory usage: {round(pdf.estimated_size('mb'),2)} MB")